In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-plot

In [ ]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer, StandardScaler
import h5py
from keras import callbacks
from sklearn.metrics import plot_confusion_matrix, classification_report
import matplotlib.pyplot as plt
import scikitplot as skplt
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import h5py
from sklearn.model_selection import KFold,cross_val_score
kfold=KFold(5,True,3)
cvscores = []

In [ ]:
#load training dataset
# Name for the column  or Features Map
columns_to_named = ["frameNumber", "frameTime", "frameLen", "ethSrc", "ethDst",
       "ipSrc", "ipDst", "ipProto", "ipLen", "tcpLen", "tcpSrcport",
       "tcpDstport", "Value", "normality"]
    
    # Read the Dataset and Rename the Column
df = pd.read_csv("/content/drive/MyDrive/Preprocessed_data.csv", header=0,names=columns_to_named)

col_norm = ['frameNumber', 'frameTime', 'frameLen', 'ethSrc', 'ethDst',
       'ipSrc', 'ipDst', 'ipProto', 'ipLen', 'tcpLen', 'tcpSrcport',
       'tcpDstport', 'Value']
    
    # Normalization using Custom Lambda Function or Min-max 

X = df[col_norm]
y = df['normality']
print(X.shape)
print(y.shape)

from sklearn.preprocessing import label_binarize
Y = label_binarize(y, classes=[0, 1, 2, 3, 4, 5])
n_classes = Y.shape[1]
print(n_classes)

(477426, 13)
(477426,)
6


In [ ]:
sc = StandardScaler()
for train, test in kfold.split(X,y):

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    x_train = np.array(X_train)
    x_test = np.array(X_test)
    y_train1 = np.array(y_train)
    y_test1 = np.array(y_test)

    y_train= to_categorical(y_train1)
    y_test= to_categorical(y_test1)

    model = Sequential()
    model.add(LSTM(16,input_dim=13, return_sequences=True))  # try using a GRU instead, for fun
    model.add(Dropout(0.4))
    model.add(LSTM(16, return_sequences=True))  # try using a GRU instead, for fun
    model.add(Dropout(0.4))
    model.add(LSTM(16, return_sequences=False))  # try using a GRU instead, for fun
    model.add(Dropout(0.4))
    model.add(Dense(6))
    model.add(Activation('softmax'))
    #model.summary()

    model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy','mae'])
    history = model.fit(X_train, y_train, epochs = 10, batch_size=64, verbose=1,shuffle=False)
    scores = model.evaluate(X_test, y_test, verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100)) 
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/10
5968/5968 [==============================] - 30s 5ms/step - loss: 0.4179 - accuracy: 0.7840 - mae: 0.0875
Epoch 2/10
5968/5968 [==============================] - 29s 5ms/step - loss: 0.2781 - accuracy: 0.8595 - mae: 0.0592
Epoch 3/10
5968/5968 [==============================] - 29s 5ms/step - loss: 0.2297 - accuracy: 0.9094 - mae: 0.0465
Epoch 4/10
5968/5968 [==============================] - 28s 5ms/step - loss: 0.1963 - accuracy: 0.9299 - mae: 0.0381
Epoch 5/10
5968/5968 [==============================] - 30s 5ms/step - loss: 0.1549 - accuracy: 0.9549 - mae: 0.0275
Epoch 6/10
5968/5968 [==============================] - 28s 5ms/step - loss: 0.1347 - accuracy: 0.9614 - mae: 0.0234
Epoch 7/10
5968/5968 [==============================] - 29s 5ms/step - loss: 0.1227 - accuracy: 0.9649 - mae: 0.0211
Epoch 8/10
5968/5968 [==============================] - 28s 5ms/step - loss: 0.1117 - accuracy: 0.9682 - mae: 0.0189
Epoch 9/10
5968/5968 [==============================] - 29s 5ms/

In [ ]:
target_names = ['Normal', 'WrongSetup', 'DDoS', 'Probing', 'Scan', 'MITM']

In [ ]:
### train_accuracy
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)
import sklearn.metrics as metrics

expected = y_train
predicted = model.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)

print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC_Train:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      Normal       1.00      0.99      0.99     63266
  WrongSetup       0.99      1.00      0.99     65753
        DDoS       1.00      1.00      1.00     63212
     Probing       1.00      0.99      1.00     63179
        Scan       1.00      0.99      0.99     63278
        MITM       0.98      1.00      0.99     63252

   micro avg       0.99      0.99      0.99    381940
   macro avg       0.99      0.99      0.99    381940
weighted avg       0.99      0.99      0.99    381940
 samples avg       0.99      0.99      0.99    381940

Accuracy_Train: 0.9946640833638791
Precision_Train: 0.994741981546926
Recall_Train: 0.9946640833638791
f1-score_Train: 0.9946729467295278
ROC_AUC_Train: 0.9996920736088225


In [ ]:
### test_accuracy
expected = y_test
predicted = model.predict(X_test)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)
print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
print("ROC_AUC:",metrics.roc_auc_score(expected, predicted,multi_class="ovr",average='weighted'))

              precision    recall  f1-score   support

      Normal       1.00      0.99      0.99     15769
  WrongSetup       0.99      1.00      0.99     16532
        DDoS       1.00      1.00      1.00     15808
     Probing       1.00      0.99      1.00     15823
        Scan       1.00      0.99      0.99     15774
        MITM       0.98      1.00      0.99     15780

   micro avg       0.99      0.99      0.99     95486
   macro avg       0.99      0.99      0.99     95486
weighted avg       0.99      0.99      0.99     95486
 samples avg       0.99      0.99      0.99     95486

Accuracy: 0.994815993967702
Precision: 0.9948757612199095
Recall: 0.994815993967702
f1-score: 0.9948180171435814
ROC_AUC: 0.9996963838665549
